# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bamboo flat,11.7000,92.7167,300.20,89,100,1.03,IN,1699925593
1,1,yellowknife,62.4560,-114.3525,267.44,86,100,3.09,CA,1699925593
2,2,port-aux-francais,-49.3500,70.2167,279.86,97,100,15.92,TF,1699925594
3,3,yuzhno-kurilsk,44.0306,145.8556,275.58,80,77,10.27,RU,1699925077
4,4,usogorsk,63.4106,48.6872,272.41,94,100,3.27,RU,1699925036


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    color = "City",
    alpha =.60
)

# Display the map
map_plot_1
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
# Max temp lower than 95 f and higher than 62 f
city_data_df = pd.read_csv("output_data/cities.csv")
def k_to_f(kelvin):
    return (kelvin - 273.15) * (9/5) + 32
city_data_df["Max Temp"] = city_data_df["Max Temp"].apply(k_to_f)
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 95) & (city_data_df["Max Temp"] > 79)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 50)]
city_data_df= city_data_df.loc[(city_data_df["Country"] != 'CN') & (city_data_df["Country"] != 'RU')]
city_data_df = city_data_df.loc[(city_data_df["Humidity"] < 80)]
city_data_df.dropna()
print(city_data_df)


     City_ID                          City      Lat       Lng  Max Temp  \
12        12        hawaiian paradise park  19.5933 -154.9731    81.914   
22        22                          kone -21.0595  164.8658    82.886   
40        40                   west island -12.1568   96.8225    80.582   
99        99                        labuan   5.2767  115.2417    86.360   
110      110                     kavaratti  10.5669   72.6420    81.302   
172      172                        mahina -17.5065 -149.4890    85.766   
175      175                          daru  -9.0763  143.2092    84.290   
189      189      rio verde de mato grosso -18.9181  -54.8442    87.260   
190      190                cabo san lucas  22.8909 -109.9124    82.760   
207      207                       kencong  -8.2833  113.3667    85.964   
257      257                      mazatlan  23.2167 -106.4167    82.904   
271      271                       pio xii  -3.8325  -45.1525    79.286   
294      294             

### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,hawaiian paradise park,US,19.5933,-154.9731,70,
22,kone,NC,-21.0595,164.8658,40,
40,west island,CC,-12.1568,96.8225,78,
99,labuan,MY,5.2767,115.2417,79,
110,kavaratti,IN,10.5669,72.6420,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'filter' : '',
    'apiKey':geoapify_key,
    'bias': ''
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon}, {lat}, {radius}'
    params["bias"] = f'proximity: {lon}, {lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
labuan - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
daru - nearest hotel: No hotel found
rio verde de mato grosso - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
kencong - nearest hotel: No hotel found
mazatlan - nearest hotel: No hotel found
pio xii - nearest hotel: No hotel found
abu dhabi - nearest hotel: No hotel found
boa vista - nearest hotel: No hotel found
champerico - nearest hotel: No hotel found
simplicio mendes - nearest hotel: No hotel found
gorontalo - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
jeddah - nearest hotel: No hotel found
ghanzi - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
sao raimundo das man

,City,Country,Lat,Lng,Humidity,Hotel Name
12,hawaiian paradise park,US,19.5933,-154.9731,70,No hotel found
22,kone,NC,-21.0595,164.8658,40,No hotel found
40,west island,CC,-12.1568,96.8225,78,No hotel found
99,labuan,MY,5.2767,115.2417,79,No hotel found
110,kavaratti,IN,10.5669,72.6420,71,No hotel found
172,mahina,PF,-17.5065,-149.4890,74,No hotel found
175,daru,PG,-9.0763,143.2092,59,No hotel found
189,rio verde de mato grosso,BR,-18.9181,-54.8442,31,No hotel found
190,cabo san lucas,MX,22.8909,-109.9124,62,No hotel found
207,kencong,ID,-8.2833,113.3667,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display
# I was unable to find a hotel for any destination, i tested with different conditions for the city_data_df and still was 
#coming up with no hotels found
# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
# YOUR CODE HERE
plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)